In [1]:
import sys
sys.path.append('/workspace/lxmert/src')
from lxrt.modeling import *
import torch
from torch.cuda.amp import GradScaler, autocast
import easydict
from collections import OrderedDict
import pickle as pkl
from torch.utils.data import Dataset,DataLoader
import numpy as np
import os
import os
from tqdm.auto import tqdm
import transformers
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="5"
import time

In [2]:
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')

In [3]:
loss_weight = 0.05
iou_threshold = 0.1

In [4]:
ious = pkl.load(open('obj_ious.pkl', 'rb'))

In [5]:
mode = f'lxmert_sequence_encoder_outsegs_obj_aware_{iou_threshold}_frame_seq_contrast_lxmert{loss_weight}_afterlstm'

In [6]:
class segmentDataset(Dataset):
    """Segment dataset."""
    def __init__(self, vidlistpkl, feat_base, cook2_IVD_dir ="/workspace/evidence_retrieval/COOK2_IVD", mode = 'train' ):
        with open(os.path.join(vidlistpkl), "rb") as f:
            vid_pkl = pkl.load(f)
            
        self.vid_list = vid_pkl
        self.feat_base = feat_base
        with open(os.path.join(cook2_IVD_dir, f"pkl/{mode}.pkl"), "rb") as f:
            self.vid_pkl = pkl.load(f)
    def __len__(self):
        return len(self.vid_list)

    def __getitem__(self, idx):
        

        vid = self.vid_list[idx]
        with open(os.path.join(self.feat_base, vid+'.pkl'), 'rb') as fp:
                  item = pkl.load(fp)
        visn_feats, encode, temporal_label, label = item
        query = self.vid_pkl[vid]["query"]
        
        
        
        return {"vid": vid, "visn_feats":visn_feats, 'encode':encode, 'temporal_label':temporal_label,'label':label, 'query':query}

In [7]:
import os
import pickle as pkl
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from statistics import mean
from tqdm import tqdm
import csv
import sys
import time
import numpy as np
import base64
cook2_IVD_dir = "/workspace/COOK2_IVD"


class segmentDataset(Dataset):
    """Segment dataset."""
    def __init__(self, vidlistpkl, feat_base, cook2_IVD_dir ="/workspace/COOK2_IVD", mode = 'train' ):
        with open(os.path.join(vidlistpkl), "rb") as f:
            vid_pkl = pkl.load(f)
            
        self.vid_list = vid_pkl
        self.feat_base = feat_base
        with open(os.path.join(cook2_IVD_dir, f"pkl/{mode}.pkl"), "rb") as f:
            self.vid_pkl = pkl.load(f)
    def __len__(self):
        return len(self.vid_list)

    def __getitem__(self, idx):
        

        vid = self.vid_list[idx]
        with open(os.path.join(self.feat_base, vid+'.pkl'), 'rb') as fp:
                  item = pkl.load(fp)
        visn_feats,encode,label,(trans_input_ids,trans_token_type_ids,trans_attention_mask ) = item
        query = self.vid_pkl[vid]["query"]
        
        
        
        return {"vid": vid, "visn_feats":visn_feats, 'encode':encode, 'label':label, 'query':query,\
               'trans_input_ids':trans_input_ids, 'trans_token_type_ids':trans_token_type_ids,\
               'trans_attention_mask':trans_attention_mask}

In [8]:
segment_dataset = segmentDataset('newsplit_train_vid_list.pkl','feat_dump_outsegs')
valid_dataset = segmentDataset('newsplit_valid_vid_list.pkl','feat_dump_outsegs', mode = 'vid')
test_dataset = segmentDataset('newsplit_test_vid_list.pkl','feat_dump_outsegs', mode = 'test')

In [9]:
class sequence_frame_encoder(torch.nn.Module):
    def __init__(self, config, sequence_encoder = 'LSTM'):
        super(sequence_frame_encoder, self).__init__()
        self.frame_encoder = LXRTFeatureExtraction(config)
        state_dict_path = os.path.join('lxmert', 'snap', 'pretrained', 'model_LXRT.pth') 
        state_dict = torch.load(state_dict_path)
        new_state_dict = OrderedDict()
        for key, value in state_dict.items():
            splittedkey = key.split('.')
            if 'bert' in splittedkey:
                newkey  = '.'.join(splittedkey[splittedkey.index('bert')+1:])
            else:
                newkey  = '.'.join(splittedkey[splittedkey.index('module')+1:])
            new_state_dict[newkey] = value
        self.frame_encoder.load_state_dict(new_state_dict, strict=False)
        del state_dict
        del new_state_dict
        if sequence_encoder == 'LSTM':
            self.sequence_encoder = torch.nn.LSTM(768, hidden_size = 768//2, batch_first = True, bidirectional = True)
            self.sequence_fc = torch.nn.Linear(768, 768)
            self.contrast_fc1 = torch.nn.Linear(768,768)
            self.contrast_fc2 = torch.nn.Linear(768,768)
            
        #transformer later
        #else:
        #    self.sequence_encoder = torch.nn.TransformerEncoderLayer(d_model = 768,nhead = 12,num_encoder_layers = 1,
        #                                                             dim_feedforward=3072, activation == "gelu", batch_first = True )
    def forward(self,visn_feats, trans_input_ids,trans_token_type_ids,trans_attention_mask ):
        frame_feats = self.frame_encoder(trans_input_ids, 
                                        token_type_ids = trans_token_type_ids,
                                        attention_mask = trans_attention_mask,
                                        visual_feats = visn_feats
                                       )
        frame_feats = frame_feats[1]        
        frame_contrast1 = self.contrast_fc1(frame_feats)
        frame_contrast1 = frame_contrast1.squeeze()
        frame_feats = frame_feats.unsqueeze(0)
        frame_feats,(_,_) = self.sequence_encoder(frame_feats)
        frame_contrast2 = self.contrast_fc1(frame_feats)
        frame_contrast2 = frame_contrast2.squeeze()
        frame_contrast = torch.cat([frame_contrast1, frame_contrast2], dim = -1)
        #frame_contrast = self.contrast_fc(frame_feats)
        #frame_contrast = frame_contrast.squeeze()
        frame_feats = self.sequence_fc(frame_feats)
        frame_feats = frame_feats.squeeze()
        return frame_feats, frame_contrast


In [10]:
config = BertConfig('bert_config.json')
"""image_encoder = LXRTImageModel(config)
state_dict_path = os.path.join('lxmert', 'snap', 'pretrained', 'model_LXRT.pth') 
state_dict = torch.load(state_dict_path)
new_state_dict = OrderedDict()
for key, value in state_dict.items():
    splittedkey = key.split('.')
    if 'bert' in splittedkey:
        newkey  = '.'.join(splittedkey[splittedkey.index('bert')+1:])
    else:
        newkey  = '.'.join(splittedkey[splittedkey.index('module')+1:])
    new_state_dict[newkey] = value
image_encoder.load_state_dict(new_state_dict, strict=False)
image_encoder.cuda().train()"""
print()

In [11]:
config = BertConfig('bert_config.json')
"""frame_encoder = LXRTFeatureExtraction(config)
state_dict_path = os.path.join('lxmert', 'snap', 'pretrained', 'model_LXRT.pth') 
state_dict = torch.load(state_dict_path)
new_state_dict = OrderedDict()
for key, value in state_dict.items():
    splittedkey = key.split('.')
    if 'bert' in splittedkey:
        newkey  = '.'.join(splittedkey[splittedkey.index('bert')+1:])
    else:
        newkey  = '.'.join(splittedkey[splittedkey.index('module')+1:])
    new_state_dict[newkey] = value
frame_encoder.load_state_dict(new_state_dict, strict=False)"""
frame_encoder = sequence_frame_encoder(config)
frame_encoder.cuda()
#frame_encoder = nn.DataParallel(frame_encoder, device_ids=[0,1])
print()

LXRT encoder with 12 l_layers, 5 x_layers, and 0 r_layers.



In [12]:
def nearest_trans_idx(frame_idx, trans_idx):
    return np.argmin(abs(trans_idx-frame_idx))

img_h, img_w = img_info['img_h'], img_info['img_w']
boxes = boxes.copy()
boxes[:, (0, 2)] /= img_w
boxes[:, (1, 3)] /= img_h

In [13]:
from transformers import BertModel, BertTokenizer
lang_encoder = BertModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
lang_encoder.cuda()
#lang_encoder = nn.DataParallel(lang_encoder, device_ids=[0,1])
print()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [14]:
from lxrt.optimization import BertAdam
epoch = 100
batch_per_epoch = len(segment_dataset)
t_total = int(batch_per_epoch * epoch)
warmup_ratio = 0.05
warmup_iters = int(t_total * warmup_ratio)
optim = BertAdam(list(frame_encoder.parameters()) + list(lang_encoder.parameters()), lr=1e-4, warmup=warmup_ratio, t_total=t_total)
loss_fn = torch.nn.CrossEntropyLoss(ignore_index=-1)
scaler = GradScaler()

In [15]:
mode 

'lxmert_sequence_encoder_outsegs_obj_aware_0.1_frame_seq_contrast_lxmert0.05_afterlstm'

batch = segment_dataset[0]

    vid, visn_feats, encode, label = batch['vid'], batch["visn_feats"],batch['encode'],batch['label']
    frame_feats, box_feats = visn_feats
    foodname = batch['query']
    trans_input_ids,trans_token_type_ids,trans_attention_mask  = batch['trans_input_ids'],batch['trans_token_type_ids'],batch['trans_attention_mask']
    label = label.cuda()
    visn_feats = frame_feats.cuda(), box_feats.cuda()
    trans_input_ids = trans_input_ids.cuda()
    trans_token_type_ids = trans_token_type_ids.cuda()
    trans_attention_mask = trans_attention_mask.cuda()
    encode = encode.cuda()
    frame_feats = frame_encoder(visn_feats, trans_input_ids, 
                                        trans_token_type_ids,trans_attention_mask
                                       )

In [16]:
floss_fn = torch.nn.BCEWithLogitsLoss(weight=None, size_average=None, reduce=None, pos_weight=None)

cos = nn.CosineSimilarity(dim=0, eps=1e-6)

contrast_feat = frame_feats[1]
frame_contrast_output = torch.matmul(contrast_feat, contrast_feat.transpose(1,0))

frame_contrast_output = m(frame_contrast_output)

label

contrast_label = []
for i in range(len(label)):
    c_label = []
    for j in range(len(label)):
        c_label.append(int(label[i]==label[j]) if label[i]>-0.5 else 0)
    contrast_label.append(c_label)
contrast_label = torch.tensor(contrast_label).float().cuda()

loss_output = loss(frame_contrast_output, contrast_label)
loss_output = loss_output*mask

torch.mean(loss_output)

mask = (label>-0.5).float()
mask = torch.matmul(mask.unsqueeze(-1), mask.unsqueeze(0))

In [17]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter(mode)

In [18]:
def retrieval_forward(batch, frame_encoder, text_encoder, compute_loss = True):
    vid, visn_feats, encode, label = batch['vid'], batch["visn_feats"],batch['encode'],batch['label']
    frame_feats, box_feats = visn_feats
    foodname = batch['query']
    trans_input_ids,trans_token_type_ids,trans_attention_mask  = batch['trans_input_ids'],batch['trans_token_type_ids'],batch['trans_attention_mask']
    label = label.cuda()
    visn_feats = frame_feats.cuda(), box_feats.cuda()
    trans_input_ids = trans_input_ids.cuda()
    trans_token_type_ids = trans_token_type_ids.cuda()
    trans_attention_mask = trans_attention_mask.cuda()
    encode = encode.cuda()
    frame_feats,contrast_feat = frame_encoder(visn_feats, trans_input_ids, 
                                        trans_token_type_ids,trans_attention_mask
                                       )
    pooled_output = frame_feats
    output = lang_encoder(encode)
    sequence_output, lang_pooled_output = output[0], output[1]
    dotoutput = torch.matmul(pooled_output, lang_pooled_output.transpose(1,0))
    if compute_loss:
        loss = loss_fn(dotoutput, label)
        frame_contrast_output = torch.matmul(contrast_feat, contrast_feat.transpose(1,0))
        #frame_contrast_output = sigmoid(frame_contrast_output)
        contrast_label = []
        for i in range(len(label)):
            c_label = []
            for j in range(len(label)):
                c_label.append(int(label[i]==label[j]) if label[i]>-0.5 else 0)
            contrast_label.append(c_label)
        contrast_label = torch.tensor(contrast_label).float().cuda()
        mask = (label>-0.5).float()
        mask = torch.matmul(mask.unsqueeze(-1), mask.unsqueeze(0))
        fcontrast_loss = floss_fn(frame_contrast_output, contrast_label)
        fcontrast_loss = torch.mean(fcontrast_loss*mask) 
        return dotoutput, loss+loss_weight*fcontrast_loss
    else:
        return dotoutput

In [19]:
min_loss = -10000
sampling_num = 3
for epoch in tqdm(range(epoch)):
    frame_encoder.train()
    lang_encoder.train()
    for iter_num in range(len(segment_dataset)):
        with autocast():
            btime_b = time.time()
            batch = segment_dataset[iter_num]
            #print(batch['encode'].shape,batch['trans_input_ids'].shape )
            optim.zero_grad()
            dotoutput, loss = retrieval_forward(batch, frame_encoder, lang_encoder,compute_loss = True)
            writer.add_scalar('Loss/train',loss.detach().cpu(), epoch*len(segment_dataset)+iter_num)
        scaler.scale(loss).backward()
        torch.nn.utils.clip_grad_norm_(list(frame_encoder.parameters()) + list(lang_encoder.parameters()),1 )
        scaler.step(optim)
        scaler.update()
        torch.cuda.empty_cache()
    with torch.no_grad():
        frame_encoder.eval()
        lang_encoder.eval()
        ks = [1,3,5]
        recalls = {k:[] for k in ks}
        losses = 0
        for iter_num in range(len(valid_dataset)):
            batch = valid_dataset[iter_num]
            dotoutput, loss = retrieval_forward(batch, frame_encoder,  lang_encoder,compute_loss = True)
            losses += loss
            label = list(batch['label'].numpy())
            for k in ks:
                        pred = np.argsort(-1*dotoutput.detach().cpu().numpy(), axis = -1)[:,:k].squeeze()
                        recallatk = 0
                        examples = 0
                        for i, gt in enumerate(label):
                            if k > 1:
                                if gt == -1:
                                    continue
                                if gt in pred[i]:
                                    recallatk +=1
                                examples += 1
                            else:
                                if gt == -1:
                                    continue
                                if gt == pred[i]:
                                    recallatk +=1
                                examples += 1
                        recallatk = recallatk/examples
                        recalls[k].append(recallatk)
        losses = losses/len(valid_dataset)
        writer.add_scalar('Loss/valid',losses.cpu(), epoch)
        print(epoch, losses)
        for k in ks:
            print( k, sum(recalls[k])/len(recalls[k]))
            writer.add_scalar(f'Recall/valid{k}',sum(recalls[k])/len(recalls[k]), epoch)
        if min_loss<sum(recalls[1])/len(recalls[1]):
                    min_loss = sum(recalls[1])/len(recalls[1])
                    torch.save(lang_encoder.state_dict(), os.path.join(f'{mode}','lang_encoder_ori_best_eval_loss.pth'.format(epoch)))
                    torch.save(frame_encoder.state_dict(), os.path.join(f'{mode}','image_encoder_ori_best_eval_loss.pth'.format(epoch)))
        for iter_num in range(len(test_dataset)):
            batch = test_dataset[iter_num]
            dotoutput, loss = retrieval_forward(batch, frame_encoder, lang_encoder, compute_loss = True)
            losses += loss
            label = list(batch['label'].numpy())
            for k in ks:
                        pred = np.argsort(-1*dotoutput.detach().cpu().numpy(), axis = -1)[:,:k].squeeze()
                        recallatk = 0
                        examples = 0
                        for i, gt in enumerate(label):
                            if k > 1:
                                if gt == -1:
                                    continue
                                if gt in pred[i]:
                                    recallatk +=1
                                examples += 1
                            else:
                                if gt == -1:
                                    continue
                                if gt == pred[i]:
                                    recallatk +=1
                                examples += 1
                        recallatk = recallatk/examples
                        recalls[k].append(recallatk)
        losses = losses/len(valid_dataset)
        writer.add_scalar('Loss/test',losses.cpu(), epoch)
        print(epoch, losses)
        for k in ks:
                print( k, sum(recalls[k])/len(recalls[k]))
                writer.add_scalar(f'Recall/test{k}',sum(recalls[k])/len(recalls[k]), epoch)
            

  0%|          | 0/100 [00:00<?, ?it/s]/workspace/lxmert/src/lxrt/optimization.py:142: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  ../torch/csrc/utils/python_arg_parser.cpp:1005.)
  next_m.mul_(beta1).add_(1 - beta1, grad)


0 tensor(2.1175, device='cuda:0')
1 0.20694159842593385
3 0.514609537812975
5 0.7585385703036757


  1%|          | 1/100 [10:30<17:19:58, 630.29s/it]

0 tensor(0.9259, device='cuda:0')
1 0.20466441984772848
3 0.5143639917360101
5 0.7520634952620809
1 tensor(2.0310, device='cuda:0')
1 0.2082063267045154
3 0.5195893505458673
5 0.7579717974406486


  2%|▏         | 2/100 [20:34<16:56:42, 622.48s/it]

...

97 tensor(1.0136, device='cuda:0')
1 0.5507528628403177
3 0.8471134020125959
5 0.948892423852609
98 tensor(2.4407, device='cuda:0')
1 0.5466890203175988
3 0.8510863200680615
5 0.9506865601021182


 99%|█████████▉| 99/100 [16:31:41<09:56, 596.32s/it]

98 tensor(1.0175, device='cuda:0')
1 0.5486819297363498
3 0.8476043406935041
5 0.9485449167326047
99 tensor(2.4421, device='cuda:0')
1 0.5479518428952148
3 0.8502466512196935
5 0.9499247653510022


100%|██████████| 100/100 [16:41:33<00:00, 600.94s/it]

99 tensor(1.0205, device='cuda:0')
1 0.5501137483362526
3 0.8473539130170674
5 0.9477644358913689


In [19]:
vidpkl = pkl.load(open(os.path.join('COOK2_IVD','pkl','vid.pkl'), 'rb'))

In [20]:
frame_encoder_state_dict = torch.load(os.path.join(mode, 'image_encoder_ori_best_eval_loss.pth'))
frame_encoder.load_state_dict(frame_encoder_state_dict)
lang_encoder_state_dict = torch.load(os.path.join(mode, 'lang_encoder_ori_best_eval_loss.pth'))
lang_encoder.load_state_dict(lang_encoder_state_dict)

<All keys matched successfully>

In [22]:
with torch.no_grad():
        frame_encoder.eval()
        lang_encoder.eval()
        ks = [1,3,5]
        recalls = {k:[] for k in ks}
        losses = 0
        for iter_num in range(len(valid_dataset)):
            batch = valid_dataset[iter_num]
            dotoutput, loss = retrieval_forward(batch, frame_encoder, lang_encoder,epoch, compute_loss = True)
            losses += loss
            label = list(batch['label'].numpy())
            for k in ks:
                        pred = np.argsort(-1*dotoutput.detach().cpu().numpy(), axis = -1)[:,:k].squeeze()
                        recallatk = 0
                        examples = 0
                        for i, gt in enumerate(label):
                            if k > 1:
                                if gt == -1:
                                    continue
                                if gt in pred[i]:
                                    recallatk +=1
                                examples += 1
                            else:
                                if gt == -1:
                                    continue
                                if gt == pred[i]:
                                    recallatk +=1
                                examples += 1
                        recallatk = recallatk/examples
                        recalls[k].append(recallatk)
        losses = losses/len(valid_dataset)
        writer.add_scalar('Loss/valid',losses.cpu(), epoch)
        print(epoch, losses)
        for k in ks:
            print( k, sum(recalls[k])/len(recalls[k]))
        for iter_num in range(len(test_dataset)):
            batch = test_dataset[iter_num]
            dotoutput, loss = retrieval_forward(batch, frame_encoder, lang_encoder,epoch, compute_loss = True)
            losses += loss
            label = list(batch['label'].numpy())
            for k in ks:
                        pred = np.argsort(-1*dotoutput.detach().cpu().numpy(), axis = -1)[:,:k].squeeze()
                        recallatk = 0
                        examples = 0
                        for i, gt in enumerate(label):
                            if k > 1:
                                if gt == -1:
                                    continue
                                if gt in pred[i]:
                                    recallatk +=1
                                examples += 1
                            else:
                                if gt == -1:
                                    continue
                                if gt == pred[i]:
                                    recallatk +=1
                                examples += 1
                        recallatk = recallatk/examples
                        recalls[k].append(recallatk)
        losses = losses/len(valid_dataset)
        print(epoch, losses)
        for k in ks:
                print( k, sum(recalls[k])/len(recalls[k]))
            

99 tensor(2.7031, device='cuda:0')
1 0.5551631858077444
3 0.8598434427384077
5 0.9522972897459667
99 tensor(1.1041, device='cuda:0')
1 0.5587508731452381
3 0.8555428180545263
5 0.9489275610512035


In [20]:
retrieval_pred = {}
with torch.no_grad():
        frame_encoder.eval()
        lang_encoder.eval()
        ks = [1,3,5]
        recalls = {k:[] for k in ks}
        losses = 0
        for iter_num in tqdm(range(len(test_dataset))):
            vid_pred = {}
            batch = test_dataset[iter_num]
            dotoutput, loss = retrieval_forward(batch, frame_encoder, lang_encoder, compute_loss = True)
            losses += loss
            label = list(batch['label'].numpy())
            for k in ks:
                        pred = np.argsort(-1*dotoutput.detach().cpu().numpy(), axis = -1)[:,:k].squeeze()
                        recallatk = 0
                        examples = 0
                        for i, gt in enumerate(label):
                            if k > 1:
                                if gt == -1:
                                    continue
                                if gt in pred[i]:
                                    recallatk +=1
                                examples += 1
                            else:
                                vid_pred[2*i+1] = pred[i]
                                if gt == -1:
                                    continue
                                if gt == pred[i]:
                                    recallatk +=1
                                examples += 1
                        recallatk = recallatk/examples
                        recalls[k].append(recallatk)
            retrieval_pred[batch['vid']] = vid_pred
        losses = losses/len(valid_dataset)
        print(epoch, losses)
        for k in ks:
                print( k, sum(recalls[k])/len(recalls[k]))
with torch.no_grad():
        frame_encoder.eval()
        lang_encoder.eval()
        ks = [1,3,5]
        recalls = {k:[] for k in ks}
        losses = 0
        for iter_num in tqdm(range(len(valid_dataset))):
            vid_pred = {}
            batch = valid_dataset[iter_num]
            dotoutput, loss = retrieval_forward(batch, frame_encoder, lang_encoder, compute_loss = True)
            losses += loss
            label = list(batch['label'].numpy())
            for k in ks:
                        pred = np.argsort(-1*dotoutput.detach().cpu().numpy(), axis = -1)[:,:k].squeeze()
                        recallatk = 0
                        examples = 0
                        for i, gt in enumerate(label):
                            if k > 1:
                                if gt == -1:
                                    continue
                                if gt in pred[i]:
                                    recallatk +=1
                                examples += 1
                            else:
                                vid_pred[2*i+1] = pred[i]
                                if gt == -1:
                                    continue
                                if gt == pred[i]:
                                    recallatk +=1
                                examples += 1
                        recallatk = recallatk/examples
                        recalls[k].append(recallatk)
            retrieval_pred[batch['vid']] = vid_pred
        losses = losses/len(valid_dataset)
        print(epoch, losses)
        for k in ks:
                print( k, sum(recalls[k])/len(recalls[k]))
            

  0%|          | 0/312 [00:00<?, ?it/s]

100 tensor(1.0021, device='cuda:0')
1 0.552947379644617
3 0.8337703010705011
5 0.9445555350399873


100%|██████████| 312/312 [01:19<00:00,  3.90it/s]

100 tensor(2.4114, device='cuda:0')
1 0.5495512755325198
3 0.8515263154173973
5 0.9512850551612152


In [24]:
iou_threshold=0.05

In [25]:
retrieval_pred = {}
retrieval_label = {}
with torch.no_grad():
        frame_encoder.eval()
        lang_encoder.eval()
        ks = [1,3,5]
        recalls = {k:[] for k in ks}
        losses = 0
        for iter_num in tqdm(range(len(test_dataset))):
            vid_pred = {}
            vid_label = {}
            batch = test_dataset[iter_num]
            vid = batch['vid']
            vid_iou = ious[vid]
            dotoutput, loss = retrieval_forward(batch, frame_encoder, lang_encoder, compute_loss = True)
            losses += loss
            label = list(batch['label'].numpy())
            vid_labels = list(set(label))
            vid_labels.remove(-1)
            hard_segs = []
            for i in vid_labels:
                if i==0:
                    if vid_iou[(0,1)]<iou_threshold:
                        hard_segs.append(i)
                elif i == max(vid_labels):
                    if vid_iou[(i-1,i)]<iou_threshold:
                        hard_segs.append(i)
                else:
                    if vid_iou[(i-1,i)]<iou_threshold and vid_iou[(i,i+1)]<iou_threshold:
                        hard_segs.append(i)
            for k in ks:
                        pred = np.argsort(-1*dotoutput.detach().cpu().numpy(), axis = -1)[:,:k].squeeze()
                        recallatk = 0
                        examples = 0
                        for i, gt in enumerate(label):
                            if gt not in hard_segs:
                                continue
                            if k > 1:
                                if gt == -1:
                                    continue
                                if gt in pred[i]:
                                    recallatk +=1
                                examples += 1
                            else:
                                vid_pred[2*i+1] = pred[i]
                                vid_label[2*i+1] = gt
                                if gt == -1:
                                    continue
                                if gt == pred[i]:
                                    recallatk +=1
                                examples += 1
                        if examples<1:
                            continue
                        recallatk = recallatk/examples
                        recalls[k].append(recallatk)
            retrieval_pred[batch['vid']] = vid_pred
            retrieval_label[batch['vid']] = vid_label
        losses = losses/len(valid_dataset)
        print(epoch, losses)
        for k in ks:
                print( k, sum(recalls[k])/len(recalls[k]))

100%|██████████| 135/135 [00:38<00:00,  3.53it/s]

100 tensor(1.0021, device='cuda:0')
1 0.5516363098202698
3 0.8204740007479437
5 0.9387999926629


In [21]:
pkl.dump(retrieval_pred, open('procedure_contrast_retrieval_result.json', 'wb'))

99 tensor(2.5399, device='cuda:0')
1 0.5500848235384761
3 0.8597046990109899
5 0.9546420315049812
99 tensor(1.0750, device='cuda:0')
1 0.5493910851437317
3 0.8543261536011475
5 0.9503941514540208
class sequence_frame_encoder(torch.nn.Module):
    def __init__(self, config, sequence_encoder = 'LSTM'):
        super(sequence_frame_encoder, self).__init__()
        self.frame_encoder = LXRTFeatureExtraction(config)
        state_dict_path = os.path.join('lxmert', 'snap', 'pretrained', 'model_LXRT.pth') 
        state_dict = torch.load(state_dict_path)
        new_state_dict = OrderedDict()
        for key, value in state_dict.items():
            splittedkey = key.split('.')
            if 'bert' in splittedkey:
                newkey  = '.'.join(splittedkey[splittedkey.index('bert')+1:])
            else:
                newkey  = '.'.join(splittedkey[splittedkey.index('module')+1:])
            new_state_dict[newkey] = value
        self.frame_encoder.load_state_dict(new_state_dict, strict=False)
        del state_dict
        del new_state_dict
        if sequence_encoder == 'LSTM':
            self.sequence_encoder = torch.nn.LSTM(768, hidden_size = 768//2, batch_first = True, bidirectional = True)
            self.sequence_fc = torch.nn.Linear(768, 768)
            self.contrast_fc = torch.nn.Linear(768,768)
            
        #transformer later
        #else:
        #    self.sequence_encoder = torch.nn.TransformerEncoderLayer(d_model = 768,nhead = 12,num_encoder_layers = 1,
        #                                                             dim_feedforward=3072, activation == "gelu", batch_first = True )
    def forward(self,visn_feats, trans_input_ids,trans_token_type_ids,trans_attention_mask ):
        frame_feats = self.frame_encoder(trans_input_ids, 
                                        token_type_ids = trans_token_type_ids,
                                        attention_mask = trans_attention_mask,
                                        visual_feats = visn_feats
                                       )
        frame_feats = frame_feats[1]        
        #frame_contrast = self.contrast_fc(frame_feats)
        #frame_contrast = frame_contrast.squeeze()
        frame_feats = frame_feats.unsqueeze(0)
        frame_feats,(_,_) = self.sequence_encoder(frame_feats)
        frame_contrast = self.contrast_fc(frame_feats)
        frame_contrast = frame_contrast.squeeze()
        frame_feats = self.sequence_fc(frame_feats)
        frame_feats = frame_feats.squeeze()
        return frame_feats, frame_contrast
99 tensor(2.3990, device='cuda:0')
1 0.5509897188919297
3 0.8555678469627827
5 0.9494686560239055
99 tensor(1.0209, device='cuda:0')
1 0.5510184906637344
3 0.8487851879782635
5 0.9473827653403941
class sequence_frame_encoder(torch.nn.Module):
    def __init__(self, config, sequence_encoder = 'LSTM'):
        super(sequence_frame_encoder, self).__init__()
        self.frame_encoder = LXRTFeatureExtraction(config)
        state_dict_path = os.path.join('lxmert', 'snap', 'pretrained', 'model_LXRT.pth') 
        state_dict = torch.load(state_dict_path)
        new_state_dict = OrderedDict()
        for key, value in state_dict.items():
            splittedkey = key.split('.')
            if 'bert' in splittedkey:
                newkey  = '.'.join(splittedkey[splittedkey.index('bert')+1:])
            else:
                newkey  = '.'.join(splittedkey[splittedkey.index('module')+1:])
            new_state_dict[newkey] = value
        self.frame_encoder.load_state_dict(new_state_dict, strict=False)
        del state_dict
        del new_state_dict
        if sequence_encoder == 'LSTM':
            self.sequence_encoder = torch.nn.LSTM(768, hidden_size = 768//2, batch_first = True, bidirectional = True)
            self.sequence_fc = torch.nn.Linear(768, 768)
            self.contrast_fc = torch.nn.Linear(768,768)
            
        #transformer later
        #else:
        #    self.sequence_encoder = torch.nn.TransformerEncoderLayer(d_model = 768,nhead = 12,num_encoder_layers = 1,
        #                                                             dim_feedforward=3072, activation == "gelu", batch_first = True )
    def forward(self,visn_feats, trans_input_ids,trans_token_type_ids,trans_attention_mask ):
        frame_feats = self.frame_encoder(trans_input_ids, 
                                        token_type_ids = trans_token_type_ids,
                                        attention_mask = trans_attention_mask,
                                        visual_feats = visn_feats
                                       )
        frame_feats = frame_feats[1]        
        frame_contrast = self.contrast_fc(frame_feats)
        frame_contrast = frame_contrast.squeeze()
        frame_feats = frame_feats.unsqueeze(0)
        frame_feats,(_,_) = self.sequence_encoder(frame_feats)
        #frame_contrast = self.contrast_fc(frame_feats)
        #frame_contrast = frame_contrast.squeeze()
        frame_feats = self.sequence_fc(frame_feats)
        frame_feats = frame_feats.squeeze()
        return frame_feats, frame_contrast

In [24]:
with open('yc2_recipes.json', 'r') as fp:
    recipes = json.load(fp)

In [25]:
with open('vid2frame_indices.pkl', 'rb') as fp:
    frame_indices = pkl.load(fp)

In [24]:

#encode = encode.cuda()

In [26]:
torch.cuda.empty_cache()

In [41]:

recipe_retrieval_result = {}
test_retrieval_result_all = {}
with torch.no_grad():
    with autocast(): 
        for iter_num in range(len(segment_dataset)):
            batch = segment_dataset[iter_num]
            vid, visn_feats, encode, label = batch['vid'], batch["visn_feats"],batch['encode'],batch['label']
            frame_feats, box_feats = visn_feats
            foodname = batch['query']
            vid_frame_indices = frame_indices[vid]
            trans_input_ids,trans_token_type_ids,trans_attention_mask  = batch['trans_input_ids'],batch['trans_token_type_ids'],batch['trans_attention_mask']
            label = label.cuda()
            visn_feats = frame_feats.cuda(), box_feats.cuda()
            trans_input_ids = trans_input_ids.cuda()
            trans_token_type_ids = trans_token_type_ids.cuda()
            trans_attention_mask = trans_attention_mask.cuda()
            vid_recipe = recipes[foodname]
            #recipe_logit[vid] = {}
            flatten_recipe = []
            for recipe in vid_recipe:
                for recipe_doc in recipe['split_ins']:
                    if recipe_doc!= '':
                        flatten_recipe.append(recipe_doc)
            encode = tokenizer.batch_encode_plus(flatten_recipe, return_tensors = 'pt',padding = True)
            frame_feats,_ = frame_encoder(visn_feats, trans_input_ids, 
                                                    trans_token_type_ids,trans_attention_mask
                                                   )
            pooled_output = frame_feats
            output = lang_encoder(input_ids = encode['input_ids'].cuda(), token_type_ids = encode['token_type_ids'].cuda(),
                                  attention_mask = encode['attention_mask'].cuda())
            sequence_output, lang_pooled_output = output[0], output[1]
            dotoutput = torch.matmul(pooled_output, lang_pooled_output.transpose(1,0))
            retrieval_result = torch.topk(dotoutput,k=3, dim = -1)[1].tolist()
            vid_retrieval_result = {vid_frame_indices[i]:[flatten_recipe[j] for j in retrieval_result[i]] for i in range(len(vid_frame_indices))}
            recipe_retrieval_result[vid] = vid_retrieval_result
        for iter_num in range(len(valid_dataset)):
            batch = valid_dataset[iter_num]
            vid, visn_feats, encode, label = batch['vid'], batch["visn_feats"],batch['encode'],batch['label']
            frame_feats, box_feats = visn_feats
            foodname = batch['query']
            vid_frame_indices = frame_indices[vid]
            trans_input_ids,trans_token_type_ids,trans_attention_mask  = batch['trans_input_ids'],batch['trans_token_type_ids'],batch['trans_attention_mask']
            label = label.cuda()
            visn_feats = frame_feats.cuda(), box_feats.cuda()
            trans_input_ids = trans_input_ids.cuda()
            trans_token_type_ids = trans_token_type_ids.cuda()
            trans_attention_mask = trans_attention_mask.cuda()
            vid_recipe = recipes[foodname]
            #recipe_logit[vid] = {}
            flatten_recipe = []
            for recipe in vid_recipe:
                for recipe_doc in recipe['split_ins']:
                    if recipe_doc!= '':
                        flatten_recipe.append(recipe_doc)
            encode = tokenizer.batch_encode_plus(flatten_recipe, return_tensors = 'pt',padding = True)
            frame_feats,_ = frame_encoder(visn_feats, trans_input_ids, 
                                                    trans_token_type_ids,trans_attention_mask
                                                   )
            pooled_output = frame_feats
            output = lang_encoder(input_ids = encode['input_ids'].cuda(), token_type_ids = encode['token_type_ids'].cuda(),
                                  attention_mask = encode['attention_mask'].cuda())
            sequence_output, lang_pooled_output = output[0], output[1]
            dotoutput = torch.matmul(pooled_output, lang_pooled_output.transpose(1,0))
            retrieval_result = torch.topk(dotoutput,k=3, dim = -1)[1].tolist()
            vid_retrieval_result = {vid_frame_indices[i]:[flatten_recipe[j] for j in retrieval_result[i]] for i in range(len(vid_frame_indices))}
            recipe_retrieval_result[vid] = vid_retrieval_result
        for iter_num in range(len(test_dataset)):
            batch = test_dataset[iter_num]
            vid, visn_feats, encode, label = batch['vid'], batch["visn_feats"],batch['encode'],batch['label']
            frame_feats, box_feats = visn_feats
            foodname = batch['query']
            vid_frame_indices = frame_indices[vid]
            trans_input_ids,trans_token_type_ids,trans_attention_mask  = batch['trans_input_ids'],batch['trans_token_type_ids'],batch['trans_attention_mask']
            label = label.cuda()
            visn_feats = frame_feats.cuda(), box_feats.cuda()
            trans_input_ids = trans_input_ids.cuda()
            trans_token_type_ids = trans_token_type_ids.cuda()
            trans_attention_mask = trans_attention_mask.cuda()
            vid_recipe = recipes[foodname]
            #recipe_logit[vid] = {}
            flatten_recipe = []
            for recipe in vid_recipe:
                for recipe_doc in recipe['split_ins']:
                    if recipe_doc!= '':
                        flatten_recipe.append(recipe_doc)
            encode = tokenizer.batch_encode_plus(flatten_recipe, return_tensors = 'pt',padding = True)
            frame_feats,_ = frame_encoder(visn_feats, trans_input_ids, 
                                                    trans_token_type_ids,trans_attention_mask
                                                   )
            pooled_output = frame_feats
            output = lang_encoder(input_ids = encode['input_ids'].cuda(), token_type_ids = encode['token_type_ids'].cuda(),
                                  attention_mask = encode['attention_mask'].cuda())
            sequence_output, lang_pooled_output = output[0], output[1]
            dotoutput = torch.matmul(pooled_output, lang_pooled_output.transpose(1,0))
            retrieval_result = torch.topk(dotoutput,k=3, dim = -1)[1].tolist()
            vid_retrieval_result = {vid_frame_indices[i]:[flatten_recipe[j] for j in retrieval_result[i]] for i in range(len(vid_frame_indices))}
            recipe_retrieval_result[vid] = vid_retrieval_result
            retrieval_result_all = np.argsort(-1*dotoutput.detach().cpu().numpy())
            vid_retrieval_result_all = {vid_frame_indices[i]:[flatten_recipe[j] for j in retrieval_result_all[i]] for i in range(len(vid_frame_indices))}
            test_retrieval_result_all[vid] = vid_retrieval_result_all

In [42]:
json.dump(test_retrieval_result_all, open('lxmert_sequence_procedure_contrast_recipe_test_retrieval_result_all.json', 'w'))

In [35]:
retrieval_result

[[178, 59, 79],
 [2, 79, 82],
 [2, 79, 82],
 [2, 60, 79],
 [2, 60, 0],
 [2, 60, 82],
 [60, 2, 27],
 [0, 27, 177],
 [0, 174, 60],
 [0, 174, 60],
 [60, 0, 174],
 [60, 0, 179],
 [12, 60, 179],
 [179, 12, 60],
 [179, 12, 108],
 [108, 12, 166],
 [179, 108, 12],
 [67, 108, 46],
 [67, 46, 81],
 [67, 104, 7],
 [104, 179, 68],
 [179, 104, 68],
 [0, 179, 174],
 [174, 0, 12],
 [179, 104, 174],
 [179, 104, 81],
 [12, 67, 60],
 [12, 67, 60],
 [12, 38, 179],
 [179, 38, 12],
 [60, 38, 12],
 [60, 38, 179],
 [60, 179, 165],
 [179, 38, 104],
 [38, 12, 81],
 [67, 12, 38],
 [81, 46, 116],
 [81, 46, 42],
 [116, 46, 131],
 [46, 116, 131],
 [46, 131, 42],
 [46, 81, 131],
 [81, 46, 144],
 [81, 144, 6],
 [144, 97, 87],
 [86, 169, 160],
 [9, 47, 57],
 [9, 47, 77],
 [47, 9, 87],
 [87, 29, 47],
 [29, 87, 53],
 [29, 99, 53],
 [18, 120, 53],
 [109, 184, 48],
 [48, 184, 109],
 [184, 48, 120],
 [184, 48, 109],
 [184, 48, 120],
 [184, 120, 109],
 [30, 135, 184],
 [135, 66, 30],
 [135, 66, 38],
 [135, 38, 66],
 [135, 6

In [40]:
np.argsort(-1*dotoutput.detach().cpu().numpy())

array([[178,  59,  79, ..., 107, 109, 172],
       [  2,  79,  82, ...,  85,  53, 109],
       [  2,  79, 177, ...,  85,  53, 109],
       ...,
       [135,  66,  38, ..., 130, 156, 164],
       [135,  38,  66, ..., 130, 156, 164],
       [135,  66,  38, ..., 164,  85, 156]])

In [ ]:
dotoutput

In [39]:
dotoutput.detach().cpu().numpy()

array([[  8.7   ,   3.46  ,  10.68  , ...,  -0.415 ,   0.2246,  -4.71  ],
       [ 13.164 ,   5.08  ,  16.03  , ...,  -0.7393,   0.4844,  -7.074 ],
       [ 17.67  ,   4.477 ,  21.44  , ...,  -1.004 ,  -2.32  , -11.25  ],
       ...,
       [  0.4785,   3.332 ,  -0.2866, ...,   0.0984,   1.104 ,   5.062 ],
       [  3.555 ,   3.045 ,   3.95  , ...,   0.1044,   0.124 ,   2.402 ],
       [  4.914 ,   0.5605,   6.5   , ...,   0.7183,  -1.6045,   1.834 ]],
      dtype=float16)

In [28]:
json.dump(recipe_retrieval_result, open('lxmert_sequence_procedure_contrast_recipe_retrieval_result.json', 'w'))

In [29]:
pkl.dump(recipe_retrieval_result, open('lxmert_sequence_procedure_contrast_recipe_retrieval_result.pkl', 'wb'))

In [52]:
encode.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

Help on method forward in module transformers.models.bert.modeling_bert:

forward(input_ids: Union[torch.Tensor, NoneType] = None, attention_mask: Union[torch.Tensor, NoneType] = None, token_type_ids: Union[torch.Tensor, NoneType] = None, position_ids: Union[torch.Tensor, NoneType] = None, head_mask: Union[torch.Tensor, NoneType] = None, inputs_embeds: Union[torch.Tensor, NoneType] = None, encoder_hidden_states: Union[torch.Tensor, NoneType] = None, encoder_attention_mask: Union[torch.Tensor, NoneType] = None, past_key_values: Union[List[torch.FloatTensor], NoneType] = None, use_cache: Union[bool, NoneType] = None, output_attentions: Union[bool, NoneType] = None, output_hidden_states: Union[bool, NoneType] = None, return_dict: Union[bool, NoneType] = None) -> Union[Tuple[torch.Tensor], transformers.modeling_outputs.BaseModelOutputWithPoolingAndCrossAttentions] method of transformers.models.bert.modeling_bert.BertModel instance
    The [`BertModel`] forward method, overrides the `__call

In [62]:
len(dotoutput), len(vid_frame_indices)

(64, 64)

In [22]:
vid = batch['vid']

In [28]:
vidpkl[vid]['segments']

[(43.0, 55.0, 'add the cabbage and water to a pan'),
 (62.0, 76.0, 'add cream and butter to a pot'),
 (112.0, 120.0, 'add the cabbage to the pot of potatos'),
 (120.0, 127.0, 'add the green onions to the pot'),
 (127.0, 143.0, 'add the bacon and pepper to the pot'),
 (143.0, 151.0, 'mash the ingredients in the pot')]